# Data Processing

The focus of this notebook is to handle data processing and create various documented datasets for our models. <br>
Each code block will create a combined dataset export from the master data, as well as create training, validation, and prediction datasets for the various ML models we employ for our project.
1. Combined Dataset <br>
Filename: all_states.csv <br>
Contains all years, states, and features in a single file, including the Year feature. <br>
2. Training Dataset <br>
Filename: all_states_training.csv <br>
Contains only the training subset (e.g., 2000–2020). <br>
3. Validation Dataset <br>
Filename: all_states_validation.csv <br>
Contains only the validation subset (from the 2000–2020 split). <br>
4. Prediction Dataset <br>
Filename: all_states_prediction.csv <br>
Contains only the data for the prediction year (2024), excluding the target feature (Election Result) if simulating real-world conditions. <br>

### Dataset# 1: (all_states.csv)<br>
This dataset is providing data for all states and creating a year feature from the original excel file. <br>
The concept here is training on all states and predicting 2024

In [17]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split


def normalize_in_recession(value):
    """
    Converts 'Y'/'YES' to 1 and 'N'/'NO' to 0 for binary classification.
    Handles uppercase and lowercase values.
    """
    if isinstance(value, str):
        value = value.strip().upper()  # Normalize to uppercase to handle all cases
        if value in ['Y', 'YES']:
            return 1
        elif value in ['N', 'NO']:
            return 0
    return value  # Return the original value if it doesn't match
    
    
def normalize_election_result(value):
    """
    Converts 'Republican'/'R' to 0 and 'Democratic'/'D' to 1 for binary classification.
    Handles uppercase and lowercase values.
    """
    if isinstance(value, str):
        value = value.strip().upper()  # Normalize to uppercase to handle 'R', 'r', 'D', 'd'
        if value in ['REPUBLICAN', 'R']:
            return 0
        elif value in ['DEMOCRATIC', 'D']:
            return 1
    return value  # Return the original value if it doesn't match


def process_excel_to_csv(input_excel_path, output_csv_path, years_to_process):
    """
    Processes an Excel file with multiple sheets into a single CSV file, adding a 'Year' column and normalizing data.
    
    Args:
    - input_excel_path: Path to the raw Excel file.
    - output_csv_path: Path to save the processed CSV file.
    - years_to_process: List of sheet names (years) to process.
    """
    # Ensure the output directory exists
    output_dir = os.path.dirname(output_csv_path)
    os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

    # Initialize an empty DataFrame to hold combined data
    combined_data = pd.DataFrame()

    # Read the Excel file
    xls = pd.ExcelFile(input_excel_path)

    # Process each sheet
    for sheet_name in years_to_process:
        if sheet_name in xls.sheet_names:
            # Read the sheet into a DataFrame
            df = pd.read_excel(xls, sheet_name=sheet_name)
            
            # Normalize the Election Result column
            if 'Election Result' in df.columns:
                df['Election Result'] = df['Election Result'].apply(normalize_election_result)
            
            # Normalize the In Recession column
            if 'In Recession (Y/N)' in df.columns:
                df['In Recession (Y/N)'] = df['In Recession (Y/N)'].apply(normalize_in_recession)
            
            # Add the 'Year' column
            df['Year'] = int(sheet_name)
            
            # Append to the combined data
            combined_data = pd.concat([combined_data, df], ignore_index=True)

    # Save the combined data to CSV
    combined_data.to_csv(output_csv_path, index=False)
    print(f"Processed data saved to {output_csv_path}")
    return combined_data


def export_split_datasets(combined_data, output_dir):
    """
    Splits the combined dataset into training, validation, and prediction sets and saves them to CSV files.
    
    Args:
    - combined_data: DataFrame containing the processed data.
    - output_dir: Directory to save the split datasets.
    """
    os.makedirs(output_dir, exist_ok=True)

    # Separate the prediction data (2024)
    prediction_data = combined_data[combined_data['Year'] == 2024].drop(columns=['Election Result'])
    combined_data = combined_data[combined_data['Year'] < 2024]

    # Separate features and target
    X = combined_data.drop(columns=['Election Result'])
    y = combined_data['Election Result']

    # Split into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Save training data
    train_data = X_train.copy()
    train_data['Election Result'] = y_train
    train_data.to_csv(os.path.join(output_dir, 'all_states_training.csv'), index=False)

    # Save validation data
    val_data = X_val.copy()
    val_data['Election Result'] = y_val
    val_data.to_csv(os.path.join(output_dir, 'all_states_validation.csv'), index=False)

    # Save prediction data
    prediction_data.to_csv(os.path.join(output_dir, 'all_states_prediction.csv'), index=False)

    print(f"Split datasets saved to {output_dir}")


# Main execution block
if __name__ == "__main__":
    # Example usage
    input_excel_path = '../data_raw/raw_data.xlsx'  # Path to raw Excel file
    output_csv_path = '../data_processed/all_states/all_states.csv'  # Path for combined dataset
    output_split_dir = '../data_processed/all_states/'  # Directory for split datasets
    years_to_process = ['2024', '2020', '2016', '2012', '2008', '2004', '2000']

    print("Starting processing pipeline...")

    try:
        # Process the raw Excel file
        combined_data = process_excel_to_csv(input_excel_path, output_csv_path, years_to_process)
        print("Combined data processing completed.")
        
        # Split and export the datasets
        export_split_datasets(combined_data, output_split_dir)
        print("Split datasets saved successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")

Starting processing pipeline...
Processed data saved to ../data_processed/all_states/all_states.csv
Combined data processing completed.
Split datasets saved to ../data_processed/all_states/
Split datasets saved successfully.


### Dataset# 2: (reliable_states.csv)<br>
This dataset is providing data for only states that predictably and reliably vote one way or the other, and creating a year feature from the original excel file. <br>
The concept here is training on only states that are hard democrat or republican, and predicting 2024